In [1]:
import sqlite3
import pandas as pd

In [2]:
movies_con = sqlite3.connect("movies.db")

In [4]:
pd.read_sql_query("SELECT * FROM sqlite_master WHERE type='table';", movies_con)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,movies,movies,3059,"CREATE TABLE ""movies"" (\n""movie_id"" INTEGER,\n..."
2,table,ratings,ratings,3740,"CREATE TABLE ""ratings"" (\n""rating_id"" INTEGER,..."


In [8]:
ratings = pd.read_sql("SELECT * FROM ratings",con=movies_con)
ratings.set_index('rating_id',inplace=True)
ratings.head()

,movie_id,user_id,rating
rating_id,,,
1,1,0,3.5
2,2,0,3.5
3,4,0,3
4,5,0,4
5,8,0,3.0


In [10]:
movies = pd.read_sql("SELECT * FROM movies",con=movies_con)
movies.set_index('movie_id',inplace=True)
movies.head()

,name,year
movie_id,,
1,10 things I hate about you,1999
2,12 Monkeys,1995
3,13 going on 30,2004
4,21 Grams,2003
5,25th hour,2002


In [15]:
pd.read_sql("SELECT * FROM movies WHERE year = 1999",con=movies_con)

,movie_id,name,year
0,1,10 things I hate about you,1999
1,17,American Beauty,1999
2,19,American Pie,1999
3,26,Austin Powers: The Spy Who Shagged Me,1999
4,27,Baby Geniuses,1999
5,34,Being John Malkovich,1999
6,36,Best Laid Plans,1999
7,37,Big Daddy,1999
8,64,Double Jeopardy,1999
9,76,Fight Club,1999


In [14]:
pd.read_sql("SELECT * FROM movies WHERE year = 1999 AND name LIKE 'A%'",con=movies_con)

,movie_id,name,year
0,17,American Beauty,1999
1,19,American Pie,1999
2,26,Austin Powers: The Spy Who Shagged Me,1999


In [17]:
pd.read_sql("SELECT * FROM movies WHERE name LIKE '%matrix%'",con=movies_con)

,movie_id,name,year
0,178,The Matrix,1999
1,179,The Matrix Reloaded,2003
2,180,The Matrix Revolutions,2003


In [20]:
# INNER JOIN to see matrix ratings
pd.read_sql(
    """
        SELECT 
            ratings.rating,ratings.user_id,movies.name 
        FROM 
            movies INNER JOIN ratings ON movies.movie_id = ratings.movie_id
        WHERE 
            movies.name LIKE '%matrix%'
    """
    ,con=movies_con
)


,rating,user_id,name
0,3.5,0,The Matrix
1,2.5,0,The Matrix Reloaded
2,0.5,1,The Matrix
3,3.5,2,The Matrix
4,0.5,2,The Matrix Reloaded
...,...,...,...
5652,3,3200,The Matrix Revolutions
5653,4.5,3201,The Matrix
5654,4.0,3201,The Matrix Reloaded
5655,3,3201,The Matrix Revolutions


In [25]:
# nested query to check rating frequencies

# INNER JOIN to see matrix ratings
pd.read_sql(
    """
        SELECT name,COUNT(*) AS rating_count FROM 
            (
                SELECT 
                    ratings.rating,ratings.user_id,movies.name 
                FROM 
                    movies INNER JOIN ratings ON movies.movie_id = ratings.movie_id
                WHERE 
                    movies.name LIKE '%matrix%'
            ) matrix_ratings
        GROUP BY name;
        
    """
    ,con=movies_con
)


,name,rating_count
0,The Matrix,2233
1,The Matrix Reloaded,1778
2,The Matrix Revolutions,1646


In [26]:
pd.read_sql(
    """
        SELECT name,rating,COUNT(*) AS rating_count FROM 
            (
                SELECT 
                    ratings.rating,ratings.user_id,movies.name 
                FROM 
                    movies INNER JOIN ratings ON movies.movie_id = ratings.movie_id
                WHERE 
                    movies.name LIKE '%matrix%'
            ) matrix_ratings
        GROUP BY name,rating;
        
    """
    ,con=movies_con
)

,name,rating,rating_count
0,The Matrix,0.0,46
1,The Matrix,0.5,42
2,The Matrix,1.0,61
3,The Matrix,1.5,71
4,The Matrix,2.0,133
5,The Matrix,2.5,214
6,The Matrix,3.0,435
7,The Matrix,3.1,1
8,The Matrix,3.3,1
9,The Matrix,3.4,1


#### Look at what the join is on: movies.movie_id = ratings.movie_id
#### If we do an inner join then an entry appears in resulting table if we have both: movie_id in movies and movie_id in ratings
#### If we do a left join, even if a movie_id doesn't exist in ratings and does exist in movies, it will show up. Vice versa for right join
#### Outer join: If there is an entry in either left column or right column, it appears in the resulting table

In [27]:
# IMPLICIT join - its a INNER JOIN by default
pd.read_sql(
    """
        SELECT 
            ratings.rating,ratings.user_id,movies.name 
        FROM 
            movies,ratings
        WHERE 
            movies.movie_id = ratings.movie_id AND movies.name LIKE '%matrix%'
    """
    ,con=movies_con
)


,rating,user_id,name
0,3.5,0,The Matrix
1,2.5,0,The Matrix Reloaded
2,0.5,1,The Matrix
3,3.5,2,The Matrix
4,0.5,2,The Matrix Reloaded
...,...,...,...
5652,3,3200,The Matrix Revolutions
5653,4.5,3201,The Matrix
5654,4.0,3201,The Matrix Reloaded
5655,3,3201,The Matrix Revolutions
